<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/TFFeedbackControlLoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import jax
jax.config.update('jax_enable_x64',True)
import jax.numpy as jnp
from plotly.subplots import make_subplots
from scipy.optimize import root
import scipy.signal as sig
from numpy.polynomial import Polynomial as P
from scipy.signal import TransferFunction as TF

In [9]:
def TF2poly(tf):
    return P(tf.num[::-1]), P(tf.den[::-1])

def poly2TF(n,d):
    return TF(n.coef[::-1],d.coef[::-1])

def TFmult(tf1, tf2):
    n1, d1 = TF2poly(tf1)
    n2, d2 = TF2poly(tf2)

    return poly2TF(n1*n2, d1*d2)

def TF_feedback(tf_f, tf_b):
    nf, df = TF2poly(tf_f)
    nb, db = TF2poly(tf_b)

    return poly2TF(nf*db, df*db + nf*nb)


def pid(Kc, taui, taud):
    return TF([Kc*taud*taui, Kc*taui, Kc],[taui,0.])


In [5]:
A1=1.
R1=2.
A2 = 1.
R2=2.
QctoQ2 = TF([1],[A1*R1, 1.])
Q2toH2 = TF([R2],[A2*R2,1.])

In [84]:
Kc=1.
taui=0.8
taud=0.
TF_fwd = TFmult(pid(Kc,taui, taud),TFmult(QctoQ2, Q2toH2))

/usr/local/lib/python3.10/dist-packages/scipy/signal/_filter_design.py:1746: BadCoefficients:

Badly conditioned filter coefficients (numerator): the results may be meaningless



In [90]:
tsim = np.linspace(0,50,500)
w=2  #rad/s
e = np.cos(w* tsim)
_, h2, _ = sig.lsim(TF_fwd, e, tsim)
fig=make_subplots()
fig.add_scatter(x=tsim, y=h2, name='output')
fig.add_scatter(x=tsim, y=e, name='input')
fig.update_layout(height=400, width=600, template='plotly_dark')

In [86]:
TF_fb = TF_feedback(TF_fwd, TF([1],[1]))

In [87]:
tplot = np.linspace(0,100, 500)
tplot, h2 = TF_fb.step(T=tplot)

In [88]:
fig=make_subplots()
fig.add_scatter(x=tplot, y=h2)
fig.update_layout(height=400, width=600, template='plotly_dark')